# 1)Чтение данных.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

In [3]:
raw_data = pd.read_csv("C:/Users/Lenovo/Downloads/Loan.csv")
true_labels = (raw_data['LoanApproved']).to_numpy() # Используем признак для классификации для проверки кластеризации
raw_data = raw_data.drop(['ApplicationDate', 'LoanApproved', 'RiskScore'], axis=1) # Удаление даты, поскольку она не имеет смыслового влияния на задачу, но добавляет лишнюю сложность, а также признаков для классификации и регрессии

numeric_feat = []
for feat in raw_data.keys():
  if ((raw_data.dtypes[feat] == 'float64' or raw_data.dtypes[feat] == 'int64') and feat != 'LoanApproved'):
      numeric_feat.append(feat)

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         20000 non-null  int64  
 1   AnnualIncome                20000 non-null  int64  
 2   CreditScore                 20000 non-null  int64  
 3   EmploymentStatus            20000 non-null  object 
 4   EducationLevel              20000 non-null  object 
 5   Experience                  20000 non-null  int64  
 6   LoanAmount                  20000 non-null  int64  
 7   LoanDuration                20000 non-null  int64  
 8   MaritalStatus               20000 non-null  object 
 9   NumberOfDependents          20000 non-null  int64  
 10  HomeOwnershipStatus         20000 non-null  object 
 11  MonthlyDebtPayments         20000 non-null  int64  
 12  CreditCardUtilizationRate   20000 non-null  float64
 13  NumberOfOpenCreditLines     200

# 2)Обработка данных.

In [6]:
for_use_data = pd.get_dummies(raw_data, columns=['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'], drop_first=True)

scaler = StandardScaler()
for_use_data[numeric_feat] = scaler.fit_transform(for_use_data[numeric_feat])
for_use_data = for_use_data.astype(float)

# 3)KMeans.

In [11]:
clustering = KMeans(n_clusters=2, random_state=0, n_init=10, tol=1e-8).fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_))} кластера(ов).")
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Это означает {correct/len(true_labels)*100}% корректность.")
#print(class_correctness)

Обнаружено 2 кластера(ов).
12985 элементов имеют корректные предсказанные метки. Это означает 64.925% корректность.


# 4)DBSCAN.

In [9]:
clustering = DBSCAN(eps=5.01, min_samples=8).fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_)) - (1) if ((-1) in clustering.labels_) else (0)} кластера(ов).") # -1 - особая метка для "шума"
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Это означает {correct/len(true_labels)*100}% корректность.")
#print(class_correctness)

Обнаружено 2 кластера(ов).
13972 элементов имеют корректные предсказанные метки. Это означает 69.86% корректность.


# 5)AgglomerativeClustering.

In [10]:
clustering = AgglomerativeClustering().fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_))} кластера(ов).")
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Это означает {correct/len(true_labels)*100}% корректность.")
#print(class_correctness)

Обнаружено 2 кластера(ов).
14401 элементов имеют корректные предсказанные метки. Это означает 72.005% корректность.


# 6)Сравнение методов.

In [13]:
table1 = [["KMeans", 64.925],
         ["DBSCAN", 69.86],
         ["AgglomerativeClustering", 72.005]]
df = pd.DataFrame(table1, columns=["Метод", "Корректность в %"])
print(df)

                     Метод  Корректность в %
0                   KMeans            64.925
1                   DBSCAN            69.860
2  AgglomerativeClustering            72.005


# 7)Вывод

Лучший результат получился с использованием метода кластеризации AgglomerativeClustering